 ### Race Conditions Caused by Implied Copies of Shared Variables in Fortran

 The following example contains a race condition, because the shared variable, which  is an array section, is passed as an actual argument to a routine that has an assumed-size  array as its dummy argument. The subroutine call passing an array section argument  may cause the compiler to copy the argument into a temporary location prior to  the call and copy from the temporary location into the original variable when the  subroutine returns. This copying would cause races in the  `parallel`  region.

In [ ]:

! name: fort_race.1f
! type: F-free
SUBROUTINE SHARED_RACE

  INCLUDE "omp_lib.h"      ! or USE OMP_LIB

  REAL A(20)
  INTEGER MYTHREAD

!$OMP PARALLEL SHARED(A) PRIVATE(MYTHREAD)

  MYTHREAD = OMP_GET_THREAD_NUM()
  IF (MYTHREAD .EQ. 0) THEN
     CALL SUB(A(1:10)) ! compiler may introduce writes to A(6:10)
  ELSE
     A(6:10) = 12
  ENDIF

!$OMP END PARALLEL

END SUBROUTINE SHARED_RACE

SUBROUTINE SUB(X)
  REAL X(*)
  X(1:5) = 4
END SUBROUTINE SUB

